In [1]:
#https://data.acgov.org/datasets/AC-HCSA::alameda-county-covid-19-cases-and-deaths-over-time-1/data
import pandas as pd
import numpy as np
import requests
#api to pull data
from arcgis.gis import GIS


/anaconda3/envs/py37/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
def i_chart_limits(sigma, x_bar, mr_bar):
    #1.128 is a known defined constant to calculate control limits
    d2 = 1.128
    ucl = x_bar + sigma*mr_bar/d2
    lcl = x_bar - sigma*mr_bar/d2
    
    return lcl, ucl



def mr_chart_limits(mr_bar):
    D4 = 3.27
    D3 = 0 
    ucl = D4*MR_BAR
    lcl = D3*MR_BAR
    
    return lcl, ucl


In [3]:
###Connect and download data from Alameda county via API
public_data_item_id = '7ea4fd9b8a1040a7b3815f2e0b5f92ba'
anon_gis = GIS()
data_item = anon_gis.content.get(public_data_item_id)
data_path = '1_data_input/api'
data_item.download(save_path=data_path)

'1_data_input/api/ACdates.csv'

In [4]:
###import downloaded data from API to dataframe
dfcov_source = pd.read_csv('1_data_input/api/ACdates.csv')
#convert columns to appropriate data types
dfcov_source['Date'] = pd.to_datetime(dfcov_source['Date'])


###import special cause manual override data
df_sc = pd.read_csv('1_data_input/special_cause_dates.csv')
df_sc['Date'] = pd.to_datetime(df_sc['Date'])



In [5]:
####select columns of interest
#Berkeley and Alameda have seprate local health juristidction (LHJ), which is why the data is split abou bteween Bk and AC
#The total of these LHJ counts give the total alameda county counts
#we are interested in using a I-MR chart on the count of total cases and deaths in Alameda county
dfcov_select = dfcov_source[['Date','BkLHJ_Cases','BkLHJ_Deaths','ACLHJ_Cases','ACLHJ_Deaths']].copy()
dfcov_select['total_cases'] = dfcov_select['BkLHJ_Cases'] + dfcov_select['ACLHJ_Cases']
dfcov_select['total_deaths'] = dfcov_select['BkLHJ_Deaths'] + dfcov_select['ACLHJ_Deaths']

#filter for only data after shelter in place order in bay area, which is 2020-03-19
#assume a stable period starts during Phase I after shelter in place
dfcov_filter = dfcov_select.loc[dfcov_select['Date'] > '2020-03-19']
dfcov = dfcov_filter[['Date', 'total_cases']].copy()

###join special cause date to flag out of control count dates
dfcov = dfcov.merge(df_sc, how='left', on='Date')



In [6]:
###Selecting the control to use
#Use I-MR chart, because subgroupings don't need to being defined. Not subgroups because # of test is unknown given current data
dfcov['moving_range'] = dfcov['total_cases'] - dfcov['total_cases'].shift(periods=1)
dfcov['moving_range'] = dfcov['moving_range'].abs()




In [7]:
###calculate control limits and lock the limits on date for when this was created
#don't use future dates, limits are locked on may 7
dfcov_locklimits = dfcov.loc[dfcov['Date'] < '2020-05-07'].copy()
#remove special causes
dfcov_locklimits = dfcov_locklimits[pd.isnull(dfcov_locklimits['special_cause_reason'])]



MR_BAR = dfcov_locklimits['moving_range'].mean()
X_BAR = dfcov_locklimits['total_cases'].mean()



In [8]:
dfcov.to_csv('test.csv')

In [9]:
X_BAR
MR_BAR

15.0

In [10]:
#calculate UCL and LCL at 3 sigma
I_LCL, I_UCL = i_chart_limits(3, X_BAR, MR_BAR)
MR_LCL, MR_UCL, = mr_chart_limits(MR_BAR)

#calculate zone limits 1, 2 for I chart
I_LCL_SIGMA_1, I_UCL_SIGMA_1 = i_chart_limits(1, X_BAR, MR_BAR)
I_LCL_SIGMA_2, I_UCL_SIGMA_2 = i_chart_limits(2, X_BAR, MR_BAR)


In [11]:
###detect special cause trends (to do)


In [12]:
#create dataframe to visualize control charts
dfcov_cc = dfcov.copy()
dfcov_cc['I_UCL'] = I_UCL
dfcov_cc['I_LCL'] = I_LCL

dfcov_cc['I_LCL_SIGMA_1'] = I_LCL_SIGMA_1
dfcov_cc['I_UCL_SIGMA_1'] = I_UCL_SIGMA_1

dfcov_cc['I_LCL_SIGMA_2'] = I_LCL_SIGMA_2
dfcov_cc['I_UCL_SIGMA_2'] = I_UCL_SIGMA_2

dfcov_cc['MR_UCL'] = MR_UCL
dfcov_cc['MR_LCL'] = MR_LCL
dfcov_cc['X_BAR'] = X_BAR
dfcov_cc['MR_BAR'] = MR_BAR


In [13]:
#flag points out of control 
dfcov_cc['I_control_flag'] = np.where((~pd.isnull(dfcov_cc['special_cause_reason'])) | (dfcov_cc['total_cases'] > dfcov_cc['I_UCL']) | (dfcov_cc['total_cases'] < dfcov_cc['I_LCL']  ) ,'out of control', 'in control')
dfcov_cc['MR_control_flag'] = np.where((dfcov_cc['moving_range'] > dfcov_cc['MR_UCL']) | (dfcov_cc['moving_range'] < dfcov_cc['MR_LCL']) ,'out of control', 'in control')


In [14]:
dfcov_cc.to_csv('2_data_output/ac_covid_control_chart.csv')